<a href="https://colab.research.google.com/github/ramayer/google-colab-examples/blob/main/Apache_Spark_with_Delta_Tables_on_Google_Colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Apache Spark with Delta Tables in Google Colab


#### Install Java

In [1]:
!apt-get install openjdk-8-jdk-headless 

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following additional packages will be installed:
  openjdk-8-jre-headless
Suggested packages:
  openjdk-8-demo openjdk-8-source libnss-mdns fonts-dejavu-extra
  fonts-ipafont-gothic fonts-ipafont-mincho fonts-wqy-microhei
  fonts-wqy-zenhei fonts-indic
The following NEW packages will be installed:
  openjdk-8-jdk-headless openjdk-8-jre-headless
0 upgraded, 2 newly installed, 0 to remove and 37 not upgraded.
Need to get 36.5 MB of archives.
After this operation, 143 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic-updates/universe amd64 openjdk-8-jre-headless amd64 8u292-b10-0ubuntu1~18.04 [28.2 MB]
Get:2 http://archive.ubuntu.com/ubuntu bionic-updates/universe amd64 openjdk-8-jdk-headless amd64 8u292-b10-0ubuntu1~18.04 [8,284 kB]
Fetched 36.5 MB in 2s (16.8 MB/s)
Selecting previously unselected package openjdk-8-jre-headless:amd64.
(Reading database

#### install Spark

In [2]:
!(wget -q --show-progress -nc https://mirrors.ocf.berkeley.edu/apache/spark/spark-3.1.2/spark-3.1.2-bin-hadoop3.2.tgz)
!tar xf spark-3.1.2-bin-hadoop3.2.tgz

spark-3.1.2-bin-had 100%[===================>] 218.23M  2.33MB/s    in 76s     


# Set Environment Variables
Set the locations where Spark and Java are installed.

In [3]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.1.2-bin-hadoop3.2"

In [4]:
!pip install --upgrade pyspark
!pip install -q findspark
!pip install -q delta

     |████████████████████████████████| 212.4 MB 71 kB/s 
     |████████████████████████████████| 198 kB 38.0 MB/s 
  Created wheel for pyspark: filename=pyspark-3.1.2-py2.py3-none-any.whl size=212880768 sha256=31cb3763c07817fd9078d1069228246f81a81b63a7e02223e637caeb745ea66e
  Stored in directory: /root/.cache/pip/wheels/a5/0a/c1/9561f6fecb759579a7d863dcd846daaa95f598744e71b02c77
Successfully built pyspark


# Start a SparkSession
This will start a local Spark session.

In [5]:
import findspark
import pyspark

MAX_MEMORY="8g"
findspark.init()
#findspark.init("/content/spark-3.0.0-preview2-bin-hadoop3.2")# SPARK_HOME
from pyspark.sql import SparkSession

#spark = SparkSession.builder.master("local[*]").getOrCreate()

spark = (pyspark.sql.SparkSession.builder.appName("MyApp") 
    .config("spark.jars.packages", "io.delta:delta-core_2.12:0.8.0") 
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") 
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog") 
    .config("spark.executor.memory", MAX_MEMORY) 
    .config("spark.driver.memory", MAX_MEMORY) 
    .enableHiveSupport() 
    .getOrCreate()        
    )

spark

In [6]:
from delta.tables import DeltaTable

In [7]:
import delta
df = spark.createDataFrame([{'hi':'hello','w':'world'}])

df.write.format('delta').mode('overwrite').option("mergeSchema", "true").save('/tmp/delta1')

In [ ]:
!df -h

# Use Spark!
That's all there is to it - you're ready to use Spark!

In [8]:
df = spark.createDataFrame([{"hello": "world"} for x in range(1000)])
df.show(3)

+-----+
|hello|
+-----+
|world|
|world|
|world|
+-----+
only showing top 3 rows



In [10]:
df.write.format("delta").saveAsTable("test_delta_table")

In [11]:
spark.sql("""
  select * from test_delta_table
""").show(3)


+-----+
|hello|
+-----+
|world|
|world|
|world|
|world|
|world|
|world|
|world|
|world|
|world|
|world|
|world|
|world|
|world|
|world|
|world|
|world|
|world|
|world|
|world|
|world|
+-----+
only showing top 20 rows

